In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.datasets import load_diabetes

data=load_diabetes()
dir(data)


['DESCR',
 'data',
 'data_filename',
 'data_module',
 'feature_names',
 'frame',
 'target',
 'target_filename']

In [62]:

data.target

X=data.data
y=data.target
data.feature_names


['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [10]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [79]:
import torch
from sklearn.preprocessing import StandardScaler

# If your data is already a numpy array (common if loaded from sklearn dataset):
# X_train and y_train are numpy arrays (no need to call .numpy())

# Standard scaling for X
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)  # no .numpy()
X_test_scaled = scaler_X.transform(X_test)

# Standard scaling for y
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).squeeze()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).squeeze()

# Convert to torch tensors
X_train = torch.from_numpy(X_train_scaled).float()
X_test = torch.from_numpy(X_test_scaled).float()
y_train = torch.from_numpy(y_train_scaled).float()
y_test = torch.from_numpy(y_test_scaled).float()


In [69]:
len(X_train),len(X_test),len(y_train),len(y_test)

(353, 89, 353, 89)

In [70]:
X_train.shape

X_train=torch.tensor(X_train, dtype=torch.float32)
X_test=torch.tensor(X_test, dtype=torch.float32)
y_train=torch.tensor(y_train, dtype=torch.float32)
y_test=torch.tensor(y_test, dtype=torch.float32)


/tmp/ipykernel_35/1130877505.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train=torch.tensor(y_train, dtype=torch.float32)
/tmp/ipykernel_35/1130877505.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test=torch.tensor(y_test, dtype=torch.float32)


In [32]:
# y=weight*x.T+ bias this is wrong

# y= x * weight.T + bias


import torch
torch.manual_seed(42)
n_in, n_out = 10, 1
limit = (6 / (n_in + n_out)) ** 0.5
weight = torch.empty(n_out, n_in).uniform_(-limit, limit)
bias = torch.zeros(n_out)

weight=weight.T
weight.shape

bias

weight1=torch.empty(n_in,n_out).uniform_(-limit,limit)
weight1.shape
weight.shape



torch.Size([10, 1])

In [36]:
import torch.nn as nn

In [49]:
import torch
import torch.nn as nn

class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()  # Corrected here!

        n_in, n_out = 10, 1
        limit = (6 / (n_in + n_out)) ** 0.5

        # Xavier initialization for weight
        self.weight = nn.Parameter(torch.empty(n_in, n_out).uniform_(-limit, limit))

        # Bias initialized to zeros
        self.bias = nn.Parameter(torch.zeros(n_out))

    def forward(self, x):
        # Use matrix multiplication (equivalent to x @ weight)
        return x @ self.weight + self.bias

model_0 = LinearRegression()
print(model_0)


LinearRegression()


In [50]:
model_0=LinearRegression()
model_0

LinearRegression()

In [72]:
res=model_0(X_test)
res

tensor([[ 2.9932e+02],
        [ 1.6250e+02],
        [ 2.6287e+02],
        [ 8.6061e+02],
        [ 1.7334e+02],
        [ 5.3432e+01],
        [ 6.0804e+02],
        [ 5.2394e+02],
        [-1.0446e+02],
        [ 7.6174e+01],
        [-8.5932e+01],
        [ 1.9056e+00],
        [-4.0050e+02],
        [ 5.1457e+02],
        [-2.7425e+01],
        [-9.2271e-01],
        [ 5.6786e+02],
        [ 6.7824e+02],
        [ 1.3779e+02],
        [ 4.8356e+02],
        [ 2.2930e+02],
        [-7.1777e+01],
        [-2.3755e+02],
        [ 3.7228e+02],
        [ 4.2113e+01],
        [ 2.3068e+02],
        [ 2.9020e+02],
        [ 2.6733e+02],
        [-4.1448e+02],
        [ 1.3503e+02],
        [ 2.6604e+02],
        [-1.3678e+02],
        [ 1.1874e+02],
        [ 3.4559e+02],
        [ 3.3550e+02],
        [ 3.7717e+02],
        [ 1.0358e+02],
        [ 1.7106e+02],
        [ 3.3699e+02],
        [-3.0617e+02],
        [-1.6037e+02],
        [ 6.5858e+00],
        [ 1.2302e+02],
        [ 3

In [103]:
loss_fn=nn.MSELoss()

optimizer=torch.optim.SGD(params=model_0.parameters(),lr=0.01)



In [100]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc
def r2_score(y_true, y_pred):
    y_true_mean = torch.mean(y_true)
    ss_tot = torch.sum((y_true - y_true_mean) ** 2)
    ss_res = torch.sum((y_true - y_pred) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2.item()


In [104]:


epochs=10000

for i in range(epochs):
    model_0.train()
    y_pred=model_0(X_train).squeeze()
    loss=loss_fn(y_pred,y_train)
    acc=r2_score(y_train,y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model_0.eval()
    with torch.inference_mode():
        y_pred1=model_0(X_test).squeeze()
        loss1=loss_fn(y_pred1,y_test)
        acc1=r2_score(y_test,y_pred1)

    if i%2000==0:
        print(f"Train loss -> {loss.item()} | test loss -> {loss1.item()} | train acc-> {acc} | test acc -> {acc1}")
        
    

Train loss -> 0.47208064794540405 | test loss -> 0.47728845477104187 | train acc-> 0.527919352054596 | test acc -> 0.4526025652885437
Train loss -> 0.47208064794540405 | test loss -> 0.47728845477104187 | train acc-> 0.527919352054596 | test acc -> 0.4526025652885437
Train loss -> 0.47208064794540405 | test loss -> 0.47728845477104187 | train acc-> 0.527919352054596 | test acc -> 0.4526025652885437
Train loss -> 0.47208064794540405 | test loss -> 0.47728845477104187 | train acc-> 0.527919352054596 | test acc -> 0.4526025652885437
Train loss -> 0.47208064794540405 | test loss -> 0.47728845477104187 | train acc-> 0.527919352054596 | test acc -> 0.4526025652885437


In [106]:
import torch
import torch.nn as nn

class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10, 1)

    def forward(self, x):
        return self.linear(x).squeeze()  # squeeze for shape

model_0 = LinearRegression()
print(model_0)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model_0.parameters(), lr=0.01)




LinearRegression(
  (linear): Linear(in_features=10, out_features=1, bias=True)
)
Train loss -> 2.0573 | test loss -> 1.4481 | train R2-> -1.0573 | test R2 -> -0.6608
Train loss -> 0.4735 | test loss -> 0.4756 | train R2-> 0.5265 | test R2 -> 0.4545
Train loss -> 0.4728 | test loss -> 0.4757 | train R2-> 0.5272 | test R2 -> 0.4544
Train loss -> 0.4724 | test loss -> 0.4761 | train R2-> 0.5276 | test R2 -> 0.4540
Train loss -> 0.4722 | test loss -> 0.4764 | train R2-> 0.5278 | test R2 -> 0.4536


In [110]:
epochs = 10000

for i in range(epochs):
    model_0.train()
    y_pred = model_0(X_train)
    loss = loss_fn(y_pred, y_train)
    acc = r2_score(y_train, y_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model_0.eval()
    with torch.inference_mode():
        y_pred1 = model_0(X_test)
        loss1 = loss_fn(y_pred1, y_test)
        acc1 = r2_score(y_test, y_pred1)

    if i % 2000 == 0:
        print(f"Train loss -> {loss.item():.4f} | test loss -> {loss1.item():.4f} | train R2-> {acc:.4f} | test R2 -> {acc1:.4f}")


Train loss -> 0.4721 | test loss -> 0.4773 | train R2-> 0.5279 | test R2 -> 0.4526
Train loss -> 0.4721 | test loss -> 0.4773 | train R2-> 0.5279 | test R2 -> 0.4526
Train loss -> 0.4721 | test loss -> 0.4773 | train R2-> 0.5279 | test R2 -> 0.4526
Train loss -> 0.4721 | test loss -> 0.4773 | train R2-> 0.5279 | test R2 -> 0.4526
Train loss -> 0.4721 | test loss -> 0.4773 | train R2-> 0.5279 | test R2 -> 0.4526
